# XPS workflow and fitting example

In this notebook a XPS measurement file from a SPECS detector (using the native SPECS .sle) is read and converted into the [NXmpes](https://manual.nexusformat.org/classes/contributed_definitions/NXmpes.html#nxmpes) NeXus standard. Additionally, a fit of a Au 4f spectrum is done.

## Create a NeXus file from measurement data

To convert the available files to the NeXus format we use the convert function readily supplied by pynxtools.

In [ ]:
from pynxtools.dataconverter.convert import convert, logger
import logging
logger.setLevel(logging.ERROR)

The input parameters are defined as follows

**input_file**: The input files for the reader. This is a xml file in specs format and a json file providing information not contained in the measurement file (e.g. user name).

**reader**: The specific reader which gets called inside the nexusparser. This is supplied in the nexusparser python code. If you create a specific reader for your measurement file it gets selecetd here. For the XPS SPECS reader it is called `xps`.

**nxdl**: The specific nxdl file which to use. For XPS this should be `NXmpes` or one of its subdefinitions of the form `NXmpes_<name>`.
    
**remove_align**: This is a special keyword for the XPS .sle reader that removes any alignment scans from the final data.

**output**: The output filename of the NeXus file.

In [ ]:
convert(input_file=["EX439_S718_Au.sle", "eln_data.yaml"],
        reader='xps',
        nxdl='NXmpes',
        remove_align=True,
        output='Au_25_mbar_O2_no_align.nxs')

## View the data with H5Web

H5Web is a tool for visualizing any data in the h5 data format. Since the NeXus format builds opon h5 it can be used to view this data as well. We just import the package and call H5Web with the output filename from the convert command above.

You can also view this data with the H5Viewer or other tools from your local filesystem.

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
H5Web("Au_25_mbar_O2_no_align.nxs")

## Analyze data

First, we need to import the necessarry packages. We use h5py for reading the NeXus file, lmfit for fitting and the class XPSRegion from the provided `xps_region.py` file.

In [2]:
import h5py
from xps_region import XPSRegion

from lmfit.models import GaussianModel

### Load data and plot

We want to load the Au 4f spectrum from the Au foil from our measurement file. Feel free to adapt to different regions in the file by changing the `MEASUREMENT` variable.

In [4]:
MEASUREMENT = "Au in vacuum__Au4f"

with h5py.File("Au_25_mbar_O2_no_align.nxs", "r") as xps_file:
    binding_energy = xps_file[f"/{MEASUREMENT}/data/energy"][:]
    cps = xps_file[f"/{MEASUREMENT}/data/data"][:]
    cps_err = xps_file[f"/{MEASUREMENT}/data/data_errors"][:]

With the loaded data we create the `au4f` `XPSRegion` containing the measurement data.

In [5]:
au4f = XPSRegion(binding_energy=binding_energy, counts=cps, counts_err=cps_err) 

There is also a convenience function in XPSRegion to directly load the data:

In [6]:
au4f = XPSRegion.load("Au_25_mbar_O2_no_align.nxs", MEASUREMENT) 

`XPSRegion` provides us a function to visualize the loaded data with

In [ ]:
au4f.plot()

### Fit data

From the preview plot we can detect two symmetric peaks which result from the spin-orbit splitting into the Au 4f5/2 and 4f3/2 regions. For illustration of the typical analysis routine, we construct two Gaussian peaks with the lmfit GaussianModel and initialize them with appropriate start values. Here we are just using initial good guesses for the start values. These, however, can eventually be deduced by data inside NOMAD as soon as enough data is available, e.g. similar to a peak detection in other XPS analysis programs. There are different peak shapes available in lmfit, such as Lorentz, Voigt, PseudoVoigt or skewed models. Please refer to the packages documentation for further details on these models and on how to use them.

In [ ]:
peak_1 = GaussianModel(prefix="Au4f52")
peak_1.set_param_hint("Au4f52_amplitude", value=1)
peak_1.set_param_hint("Au4f52_sigma", value=1)
peak_1.set_param_hint("Au4f52_center", value=84.2)

peak_2 = GaussianModel(prefix="Au4f32")
peak_2.set_param_hint("Au4f32_amplitude", value=1)
peak_2.set_param_hint("Au4f52_sigma", value=1)
peak_2.set_param_hint("Au4f32_center", value=86.9)

We can simply add the two models together to create a composite model

In [ ]:
comp = peak_1 + peak_2 

In the next step we select a region in which we want to fit the data with `fit_region(...)`, calculate a shirley baseline with `calc_baseline()`, set the fit model (`.fit_model(comp)`) and perform a fit (`.fit()`). All of this functions can also be used independently. The fit function takes the measurement uncertainties as weights to the fit function into account.

Finally, the model is plotted with the previously used `plot()` method. Since we performed a fit the plot is now extended by the baseline and fits.

In [ ]:
au4f.calc_baseline().fit_model(comp).fit().plot()

The fit result gets stored inside the `fit_result` parameter and is displayed to extract, e.g., the peak central energies. Please note that the fitting does not take the measurement uncertainties into account and the errors are simple fitting errors.

In [ ]:
au4f.fit_result.params 

We can also extract a fitting parameter shared accross different peaks, e.g. the peak central energies. This refers to the text behind the model paramters prefix, so we select `center` here to get the central energies.

In [ ]:
au4f.peak_property('center')

Typically, we are also interested in the peak areas which can be calculated with `peak_areas()`

In [ ]:
(areas := au4f.peak_areas())

and their ratios

In [ ]:
areas / areas.max()

To asses the quality of the fit the fit residual can be viewed with `plot_residual()`.

In [ ]:
au4f.plot_residual()